In [2]:
 ! pip install -q kaggle
 ! mkdir ~/.kaggle
 ! cp kaggle.json ~/.kaggle/
 ! chmod 600 ~/.kaggle/kaggle.json
 ! kaggle competitions download -c /vuppalaadithyasairam/bone-fracture-detection-using-xrays

mkdir: cannot create directory ‘/root/.kaggle’: File exists
404 Client Error: Not Found for url: https://www.kaggle.com/api/v1/competitions/data/download-all//vuppalaadithyasairam/bone-fracture-detection-using-xrays


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vuppalaadithyasairam/bone-fracture-detection-using-xrays")

print("Path to dataset files:", path)

100%|██████████| 172M/172M [00:02<00:00, 73.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/vuppalaadithyasairam/bone-fracture-detection-using-xrays/versions/1


In [4]:
import tensorflow as tf
import os
import glob

train_files = tf.data.Dataset.list_files(glob.glob(os.path.join(path, 'archive (6)', 'train', '*', '*'+'.jpg')),
                                        shuffle=True,
                                        seed=123)
val_files = tf.data.Dataset.list_files(glob.glob(os.path.join(path, 'archive (6)', 'val', '*', '*'+'.jpg')),
                                        shuffle=True,
                                        seed=123)

In [5]:
import matplotlib.pyplot as plt

for i in train_files.take(1):
  print(i)

tf.Tensor(b'/root/.cache/kagglehub/datasets/vuppalaadithyasairam/bone-fracture-detection-using-xrays/versions/1/archive (6)/train/not fractured/16-rotated2-rotated1-rotated3.jpg', shape=(), dtype=string)


In [6]:
def image_label_io(path):
  img = tf.io.read_file(path)
  img = tf.io.decode_jpeg(img, 1)
  img = tf.image.resize(img, (128, 128))
  img = tf.image.grayscale_to_rgb(img)
  img = tf.image.convert_image_dtype(img, tf.float32)

  l = tf.strings.split(path, os.path.sep)
  label = (l[-2]=='fractured')

  return (img, label)

In [7]:
def data_augment(img):
  img = tf.image.random_flip_left_right(img)
  img = tf.image.random_flip_up_down(img)
  img = tf.image.random_contrast(img, 0.9, 1.1)
  img = tf.image.random_brightness(img, 0.1)

  return img

In [8]:
def img_prep(img):
  img = tf.keras.applications.vgg16.preprocess_input(img)

  return img

In [9]:
train_set = train_files.map(image_label_io)
train_set = train_set.map(lambda x, y: (img_prep(x), y))
train_set = train_set.map(lambda x, y: (data_augment(x), y))
train_set = train_set.batch(32)
train_set.cache().prefetch(tf.data.AUTOTUNE)

val_set = val_files.map(image_label_io)
val_set = val_set.map(lambda x, y: (img_prep(x), y))
val_set = val_set.batch(32)
train_set.cache().prefetch(tf.data.AUTOTUNE)


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.bool, name=None))>

In [10]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import  Dense, Dropout, Flatten

base_model =tf.keras.applications.VGG16(include_top=False,
                                        weights="imagenet",
                                        pooling='avg')
base_model.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Using a file to schedule learning rate.

In [12]:
with open('lr.txt', 'r') as f:
  print(f.readlines()[1])

0.00001



In [13]:
import numpy as np

def read_from_file(file_name):
  def lr_setter(epoch, lr):
    with open(file_name, 'r') as f:
      if epoch < 20:
        return np.float32(f.readlines()[0])
      if epoch >= 20:
        return np.float32(f.readlines()[1])
  return lr_setter

In [14]:
scheduler = read_from_file('lr.txt')
myCallback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [15]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_set, epochs=40, validation_data=val_set, callbacks=[myCallback])


Epoch 1/40
277/277 ━━━━━━━━━━━━━━━━━━━━ 42s 122ms/step - accuracy: 0.4925 - loss: 4.7301 - val_accuracy: 0.4433 - val_loss: 1.8888 - learning_rate: 1.0000e-04
Epoch 2/40
277/277 ━━━━━━━━━━━━━━━━━━━━ 18s 66ms/step - accuracy: 0.5229 - loss: 3.7916 - val_accuracy: 0.5367 - val_loss: 1.5010 - learning_rate: 1.0000e-04
Epoch 3/40
277/277 ━━━━━━━━━━━━━━━━━━━━ 20s 65ms/step - accuracy: 0.5368 - loss: 3.0695 - val_accuracy: 0.5917 - val_loss: 1.1516 - learning_rate: 1.0000e-04
Epoch 4/40
277/277 ━━━━━━━━━━━━━━━━━━━━ 18s 66ms/step - accuracy: 0.5597 - loss: 2.4356 - val_accuracy: 0.6517 - val_loss: 0.9624 - learning_rate: 1.0000e-04
Epoch 5/40
277/277 ━━━━━━━━━━━━━━━━━━━━ 19s 67ms/step - accuracy: 0.5699 - loss: 2.0423 - val_accuracy: 0.6900 - val_loss: 0.8102 - learning_rate: 1.0000e-04
Epoch 6/40
277/277 ━━━━━━━━━━━━━━━━━━━━ 19s 67ms/step - accuracy: 0.5844 - loss: 1.7370 - val_accuracy: 0.7050 - val_loss: 0.6600 - learning_rate: 1.0000e-04
Epoch 7/40
277/277 ━━━━━━━━━━━━━━━━━━━━ 20s 64ms/st